## CLI 기반 트레이딩 백테스트

Reference: https://data.oecd.org/leadind/composite-leading-indicator-cli.htm

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (12, 6)

%matplotlib inline

import redis

cache_ip = '198.13.60.78'
cache_pw = 'da56038fa453c22d2c46e83179126e97d4d272d02ece83eb83a97357e842d065'

r = redis.StrictRedis(host=cache_ip, port=6379, password=cache_pw)

# redis keys
KOSPI_INDEX = 'I.001_INDEX'
KOSPI_LEVERAGE = '122630_OHLCV' # 코스피 2배 레버리지
KODEX_INVERSE = '114800_OHLCV' # 코스피 인버스
KODEX_INVERSE_LEVERAGE = '252670_OHLCV' # KODEX 200선물인버스2X
BOND = '114260_OHLCV' # KODEX 국고채3년

# 지수 데이터
kp = pd.read_msgpack(r.get(KOSPI_INDEX))
# ETF 데이터
kp_lvg = pd.read_msgpack(r.get(KOSPI_LEVERAGE))
kp_inv = pd.read_msgpack(r.get(KODEX_INVERSE))
kp_inv_lvg = pd.read_msgpack(r.get(KODEX_INVERSE_LEVERAGE))
bond = pd.read_msgpack(r.get(BOND))

In [23]:
# ETF 가격 정보 모두 합치기
kp_prc = kp.set_index('date')['cls_prc']
kp_lvg_prc = kp_lvg.set_index('date')['adj_prc']
kp_inv_prc = kp_inv.set_index('date')['adj_prc']
kp_inv_lvg_prc = kp_inv_lvg.set_index('date')['adj_prc']
bond_prc = bond.set_index('date')['adj_prc']

df = pd.concat([kp_prc, kp_lvg_prc, kp_inv_prc, kp_inv_lvg_prc, bond_prc], axis=1)
df.index = pd.to_datetime(df.index)
df.columns = ['KP', 'KP Leverage', 'KP Inverse', 'KP Inverse Leverage', 'Bond']

/Users/abc/.virtualenvs/data/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [42]:
# 1개월 데이터로 수정
df_m = df.resample('M').last()
df_m.index = df_m.index.to_period('M') # index의 date에서 year과 month만 나오도록 수정

In [43]:
df_m.head()

,KP,KP Leverage,KP Inverse,KP Inverse Leverage,Bond
2013-01,1961.94,12315,7575,NaN,48437
2013-02,2026.49,13290,7285,NaN,48742
2013-03,2004.89,12740,7435,NaN,49034
2013-04,1963.95,11935,7670,NaN,49129
2013-05,2001.05,12415,7515,NaN,48836


In [26]:
### csv (comma separated values) 파일 읽어오는 방법
import os

# 우선 폴더 내부에 있는 파일들 프린트해보기
os.listdir()

['기본이론 [Day 2] - Money Management.ipynb',
 '005930.xls',
 '.DS_Store',
 'Composite Leading Indicator (CLI).ipynb',
 '수급분석툴.ipynb',
 '리스크관리툴.ipynb',
 'marketsignal_bt.csv',
 'README.md',
 'sd_bt.csv',
 'GAA.ipynb',
 '.gitignore',
 '마켓시그널툴.ipynb',
 '기본이론 [Day 1] - Risk.ipynb',
 'cli.csv',
 '기본이론 [Day 3] - Portfolio Management.ipynb',
 '.ipynb_checkpoints',
 '기본이론 [Day 0] - Statistics.ipynb',
 '.git']

In [29]:
# cli.csv 파일을 열어본다
cli = pd.read_csv('cli.csv')
cli.head() # 상위 5줄 프린트

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,GBR,CLI,AMPLITUD,LTRENDIDX,M,2004-01,100.6680,NaN
1,GBR,CLI,AMPLITUD,LTRENDIDX,M,2004-02,100.7777,NaN
2,GBR,CLI,AMPLITUD,LTRENDIDX,M,2004-03,100.8075,NaN
3,GBR,CLI,AMPLITUD,LTRENDIDX,M,2004-04,100.7725,NaN
4,GBR,CLI,AMPLITUD,LTRENDIDX,M,2004-05,100.6897,NaN


In [32]:
# 불러온 location 정보들을 확인한다
locations = list(set(cli['LOCATION'])) # set으로 바꿔서 유니크한 값들만 빼낸 후 리스트로 변환한다
locations

['JPN', 'EA19', 'G-7', 'KOR', 'GBR', 'USA', 'CHN', 'OECD']

In [48]:
jpn_cli = cli[cli['LOCATION'] == 'JPN'] # 일본
ea19_cli = cli[cli['LOCATION'] == 'EA19'] # EA 19개 국가
g7_cli = cli[cli['LOCATION'] == 'G-7'] # G-7 국가들
kor_cli = cli[cli['LOCATION'] == 'KOR'] # 한국
uk_cli = cli[cli['LOCATION'] == 'GBR'] # 영국
us_cli = cli[cli['LOCATION'] == 'USA'] # 미국
chn_cli = cli[cli['LOCATION'] == 'CHN'] # 중국
oecd_cli = cli[cli['LOCATION'] == 'OECD'] # OECD 국가들

for dataframe in [jpn_cli, ea19_cli, g7_cli, kor_cli, uk_cli, us_cli, chn_cli, oecd_cli]:
    dataframe.set_index('TIME', inplace=True)
    dataframe.index = pd.to_datetime(dataframe.index).to_period('M')
    dataframe.drop(['LOCATION', 'INDICATOR', 'SUBJECT', 'MEASURE', 'FREQUENCY', 'Flag Codes'], axis=1, inplace=T)

In [49]:
jpn_cli

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,Value,Flag Codes
TIME,,,,,,,
2004-01,JPN,CLI,AMPLITUD,LTRENDIDX,M,100.51790,NaN
2004-02,JPN,CLI,AMPLITUD,LTRENDIDX,M,100.57850,NaN
2004-03,JPN,CLI,AMPLITUD,LTRENDIDX,M,100.63700,NaN
2004-04,JPN,CLI,AMPLITUD,LTRENDIDX,M,100.68320,NaN
2004-05,JPN,CLI,AMPLITUD,LTRENDIDX,M,100.72340,NaN
2004-06,JPN,CLI,AMPLITUD,LTRENDIDX,M,100.76020,NaN
2004-07,JPN,CLI,AMPLITUD,LTRENDIDX,M,100.77330,NaN
2004-08,JPN,CLI,AMPLITUD,LTRENDIDX,M,100.75690,NaN
2004-09,JPN,CLI,AMPLITUD,LTRENDIDX,M,100.72900,NaN
